In [1]:
import json
import os

import openmc
import pandas as pd
from lxml import etree
from tqdm import tqdm

from utils.resulting import DepletionResultReader

In [2]:
results_path = "results/depletion/v5/"
os.listdir(results_path)

['0', '1', '4', '6']

In [3]:
res = DepletionResultReader(results_path + os.listdir(results_path)[0] + "/depletion_results.h5", fuel_mat='2')
res.data

KeyboardInterrupt: 

In [ ]:
data = res.prepare_data()

In [ ]:
# params = json.load(open(results_path + os.listdir(results_path)[0] + "/results.json",'r'))
with open(results_path + os.listdir(results_path)[0] + "/results.json", 'r') as f:
    enr = f.read().split('"fuel_enr": ')[1].split(',')[0]
params = {
    'enr': float(enr),
    "materials": {},
    'geometry': {}
}
params

In [ ]:
pd.DataFrame(data)

In [ ]:
with open(results_path + os.listdir(results_path)[0] + '/materials.xml', 'rb') as f:
    contents = f.read()
    print(etree.fromstring(contents))

In [ ]:
tree = etree.parse(results_path + os.listdir(results_path)[0] + '/materials.xml')
tree.getroot().getchildren()

In [ ]:
for child in tree.getroot():
    print(child.get('name'), child.get('volume'), sep=": ")
    dens = child.find('density')
    print(dens.get('value'), dens.get('units'), sep=' ')

In [60]:
def get_materials_info(results_path):
    mats_path = os.path.join(results_path, "materials.xml")
    tree = etree.parse(mats_path)
    res = {
        'water': None,
        'fuel': None,
        'cladding': None
    }
    for child in tree.getroot():
        info = {
            "volume": float(child.get('volume', 0)),
            "density": float(child.find('density').get('value', 0)),
        }
        if child.get('name') == 'Water':
            res['water'] = info
        elif child.get('name') == 'Fuel':
            info['id'] = child.get('id', '2')
            res['fuel'] = info
        elif child.get('name') == 'Zircaloy-2':
            res['cladding'] = info
        else:
            print("Material ot found: ", child.get('name'))
    return res

In [61]:
get_materials_info(results_path + os.listdir(results_path)[0])

{'water': {'volume': 155.4871681372756, 'density': 1.0},
 'fuel': {'volume': 429.76987501108374, 'density': 8.3, 'id': '2'},
 'cladding': {'volume': 573.0265000147781, 'density': 6.56}}

In [62]:
params

{'enr': 3.0, 'materials': {}, 'geometry': {}}

In [64]:
def merge_context(exp_params: dict, mats: dict) -> dict:
    res = {
        'enr': exp_params['materials'].get('fuel_enr', exp_params.get('enr', 3))
    }
    for mat_name, mat_data in mats.items():
        for key, value in mat_data.items():
            res[f"{mat_name}_{key}"] = value
    return res


merge_context(params, get_materials_info(results_path + os.listdir(results_path)[0]))

{'enr': 3.0,
 'water_volume': 155.4871681372756,
 'water_density': 1.0,
 'fuel_volume': 429.76987501108374,
 'fuel_density': 8.3,
 'fuel_id': '2',
 'cladding_volume': 573.0265000147781,
 'cladding_density': 6.56}

In [65]:
def prepare_sample(res_dir: str, save_dir: str = 'neuro/data/') -> None:
    mats_data = get_materials_info(res_dir)
    # params = json.load(open(os.path.join(res_dir, "results.json"), 'r'))
    with open(results_path + os.listdir(results_path)[0] + "/results.json", 'r') as f:
        enr = f.read().split('"fuel_enr": ')[1].split(',')[0]
    params = {
        'enr': float(enr),
        "materials": {},
        'geometry': {}
    }
    context = merge_context(params, mats_data)
    fuel_id = context.pop('fuel_id', '2')
    dep_res = DepletionResultReader(os.path.join(res_dir, "depletion_results.h5"), fuel_mat=fuel_id)
    df = pd.DataFrame(dep_res.prepare_data())
    os.makedirs(save_dir, exist_ok=True)
    json.dump(context, open(os.path.join(save_dir, "context.json"), 'w'))
    df.to_csv(os.path.join(save_dir, "data.csv"))


# test_res_path = os.listdir(results_path)[0]
# prepare_sample(
#     res_dir=os.path.join(results_path, test_res_path),
#     save_dir=os.path.join('neuro/data/v2', test_res_path)
# )

In [67]:
for result_dir in tqdm(os.listdir(results_path)):
    res_dir = os.path.join(results_path, result_dir)
    save_dir = os.path.join('neuro/data/v3', result_dir)
    try:
        prepare_sample(res_dir, save_dir)
    except Exception as e:
        print(f"Failed to prepare sample: {res_dir}", e)

100%|██████████| 164/164 [1:05:10<00:00, 23.84s/it]


In [68]:
for res_dir in tqdm(os.listdir(results_path)):
    try:
        with open(results_path + res_dir + "/results.json", 'r') as f:
                enr = f.read().split('"fuel_enr": ')[1].split(',')[0]
                print(enr)
    except Exception as e:
        print(f"Failed to prepare sample: {res_dir}", e)

100%|██████████| 164/164 [00:00<00:00, 2960.33it/s]

3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
